<b>Partie 2 - Scénario 3</b>

<b>1. Création des données</b>

In [40]:
import pandas as pd

df_maisons=pd.read_csv('maisons.csv')
df_maisons.head()

,surface,nb_chambre,type,prix,cher
0,100,3,normal,300,0
1,150,4,haut standing,500,1
2,120,3,normal,400,0
3,80,2,normal,250,0
4,200,5,haut standing,600,1


In [41]:
X=df_maisons[['surface','nb_chambre','type']]
y=df_maisons['cher']

<b>2. Encodage de feature à valeurs discrètes</b>

In [42]:
from sklearn.preprocessing import OneHotEncoder

ohe=OneHotEncoder()
ohe.fit(X[['type']])
x_type_encoded=ohe.transform(X[['type']])
df_type_encoded=pd.DataFrame(x_type_encoded.toarray(), 
                                columns='type_'+ohe.categories_[0])
X=pd.concat([X,df_type_encoded], axis=1)
X.drop('type', axis=1, errors='ignore',inplace=True)
X.head()

,surface,nb_chambre,type_haut standing,type_normal
0,100,3,0.0,1.0
1,150,4,1.0,0.0
2,120,3,0.0,1.0
3,80,2,0.0,1.0
4,200,5,1.0,0.0


<b>3. Normalisation des données</b>

In [43]:
from sklearn.preprocessing import StandardScaler

ss=StandardScaler()
ss.fit(X)
X_ss=ss.transform(X)

<b>4. Réduction des dimensions</b>

In [44]:
from sklearn.decomposition import PCA

pca=PCA(n_components=2)
pca.fit(X_ss)
X_pca=pca.transform(X_ss)

<b>5. Création & Configuration du modèle</b>

In [45]:
from sklearn.linear_model import SGDClassifier

regressor=SGDClassifier(learning_rate='constant', 
                        eta0=0.00001,
                        max_iter=10000,
                        tol=0.001,
                        loss='log_loss',
                        penalty=None,
                        verbose=0)

<b>6. Entrainement du modèle</b>

In [46]:
regressor.fit(X_pca,y)

SGDClassifier(eta0=1e-05, learning_rate='constant', loss='log_loss',
              max_iter=10000, penalty=None)

<b>7. Valeurs optimales des paramètres</b>

In [47]:
w=regressor.coef_
w

array([[9.65076103e-04, 3.00402745e-05]])

In [48]:
b=regressor.intercept_
b

array([-0.00011998])

<b>8. Prédiction du prix d'une nouvelle maison</b>

In [49]:
x_new=pd.DataFrame([[400,4,'haut standing']], 
                   columns=['surface','nb_chambre','type'])
x_new

,surface,nb_chambre,type
0,400,4,haut standing


In [50]:
x_type_encoded=ohe.transform(x_new[['type']])
df_type_encoded=pd.DataFrame(x_type_encoded.toarray(), 
                                columns='type_'+ohe.categories_[0])
x_new=pd.concat([x_new,df_type_encoded], axis=1)
x_new.drop('type', axis=1, errors='ignore',inplace=True)

x_new_ss=ss.transform(x_new)

x_new_pca=pca.transform(x_new_ss)

y_new=regressor.predict(x_new_pca)

if y_new[0]==1:
    print('Maison est chère !')
else:
    print('Maison n\'est pas chère !')

Maison est chère !
